In [ ]:
import torch
from CustomPALI3.CustomPALI3 import CustomPALI3,CustomPALI3Config
from PIL import Image
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers.modeling_utils import PreTrainedModel
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
img=Image.open('test.png').convert('RGB').resize((512,512))
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
device=torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
# device=torch.device("cpu")
img = torch.from_numpy(np.array(img)).permute(2, 0, 1).unsqueeze(dim=0).to(device)
query="summarize this chart"
tonkens = tokenizer(
        query, max_length=512, padding="max_length", truncation=True,return_tensors="pt")
prompt=tonkens['input_ids'].to(device)
mask = tonkens['attention_mask'].numpy()
mask=mask==1
mask=torch.tensor(mask).to(device)
answer="This trace chart displays the \u770c's values over time, with green indicating normal values and light green indicating abnormal ones. The overshoot of the abnormal signal is approximately 57% longer than that of the normal signals. Moreover, the two signals are easily distinguishable. The amplitude of the abnormal signal is smaller than that of the normal signal."
output_text = tokenizer(
        answer, max_length=1024, padding="max_length", truncation=True,return_tensors="pt")['input_ids'].to(device)

output_text = torch.randint(0, 256, (1, 1024)).to(device)
config=CustomPALI3Config(model_name='test',
        image_size=512,
        patch_size=256,
        dim=512,
        depth=12,
        heads=32,
        enc_num_tokens=32100,
        enc_max_seq_len=1024,
        dec_num_tokens=32100,
        dec_max_seq_len=1024,
        enc_depth=12,
        enc_heads=32,
        dec_depth=12,
        dec_heads=32,
        seq_len=1024,
        device=device)
model = CustomPALI3(config)

result=model(img, prompt, output_text, mask)

# count=0
# def mul(list_):
#         init=1
#         for i in list_:
#                 init*=i
#         return init
# for name, param in model.vit_model.vit.named_parameters():
#         count+=mul(np.array(param.size()).tolist())
# print(count/1000000,"M")
# count=0
# for name, param in model.pali_model.named_parameters():
#         count+=mul(np.array(param.size()).tolist())
# print(count/1000000,"M")
# # https://huggingface.co/datasets/timm/imagenet-22k-wds
# # https://huggingface.co/datasets/conceptual_captions
# # https://huggingface.co/datasets/poloclub/diffusiondb

In [1]:
from CustomPALI3.SummaryChartDataset import SummaryChartDataset
from CustomPALI3.CustomPALI3 import CustomPALI3Config,CustomPALI3
from transformers import T5Tokenizer
import torch
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
import os
from pytorch_model_summary import summary
import numpy as np
import gc
dataset_repo=[{'dataset':'timm/imagenet-12k-wds','config':'default','type':'vision-text'},
                {'dataset':'wikimedia/wikipedia','config':'20231101.en','type':'text'},
                {'dataset':'conceptual_captions','config':'labeled','type':'vision-text'},
                {'dataset':'poloclub/diffusiondb','config':'2m_random_1m','type':'vision-text'},
                ]
def my_collate_fn(samples):
    image_batch = []
    input_batch = []
    attn_mask_batch = []
    
    batch_size=len(samples)
    
    image_batch_ = []
    input_batch_ = []
    attn_mask_batch_ = []
    for sample in samples:
        image_batch_.extend(sample['image'])
        input_batch_.extend(sample['input_ids'])
        attn_mask_batch_.extend(sample['attn_mask'])
    
    total_b=len(image_batch_)//batch_size # 14 //4   3.xx 3  
    total_b=total_b+1 if len(image_batch_)%batch_size!=0  else total_b
    for i in range(total_b):
        if (i+1)*batch_size<len(image_batch_):
            image_batch.append(torch.stack(image_batch_[i*batch_size:(i+1)*batch_size]))
            input_batch.append(torch.stack(input_batch_[i*batch_size:(i+1)*batch_size]))
            attn_mask_batch.append(torch.stack(attn_mask_batch_[i*batch_size:(i+1)*batch_size]))
        else:
            image_batch.append(torch.stack(image_batch_[i*batch_size:]))
            input_batch.append(torch.stack(input_batch_[i*batch_size:]))
            attn_mask_batch.append(torch.stack(attn_mask_batch_[i*batch_size:]))

    return {'image': image_batch, 'input_ids': input_batch,'attn_mask':attn_mask_batch}
args={
    'output_dir':'/Users/dongunyun/study/datascience/chart2text/PALI3/output',
    'lr':1e-4,
    'max_steps':1e4,
    'valid_steps':1e2,
    'num_epochs':100,
    'batch_size':4,
    'num_training_samples_per_epoch':10,
    'max_epochs':100,
    "warmup_steps":100,
    'num_workers':1,
    'num_nodes':1,
    }

tokenizer=T5Tokenizer.from_pretrained("google/flan-t5-base")
train_loader=DataLoader(SummaryChartDataset(dataset_repo,1024,tokenizer,'</s>','train'), batch_size=1, shuffle=True, num_workers=1,collate_fn=my_collate_fn)
val_loader=DataLoader(SummaryChartDataset(dataset_repo,1024,tokenizer,'</s>','validation'), batch_size=1, shuffle=True, num_workers=1,collate_fn=my_collate_fn)
config=CustomPALI3Config(version=1,model_name='test',
                    dim=1024,enc_num_tokens=32100,enc_max_seq_len=1024,
                    dec_num_tokens=32100,dec_max_seq_len=1024,enc_depth=12,enc_heads=16,dec_depth=12,dec_heads=16,seq_len=1024
                    ,device='mps',vit_fix=False)

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device =torch.device("cpu")


model=CustomPALI3(config)
model=model.from_pretrained("/Users/dongunyun/study/datascience/chart2text/PALI3/output_temp")

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dongunyun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/dongunyun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-03-03 17:47:03,787 - DEBUG - Starting new HTTPS connection (1): huggingface.co:443
2024-03-03 17:47:04,030 - DEBUG - https://huggingface.co:443 "HEAD /google/flan-t5-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. I

In [4]:
from torchvision import transforms
from PIL import Image
img=Image.open('conceptual_captions.png').convert("RGB").resize((336,336),Image.Resampling.BILINEAR )
label='Explain this picture. '

input_image_tensor = transforms.transforms.ToTensor()(img).squeeze(0)
input_image_tensor = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(input_image_tensor).unsqueeze(0).to(device)
outputs = tokenizer(label, 
                                    max_length=1024, 
                                    padding="max_length", 
                                    truncation=True,
                                    return_tensors="pt",
                                    return_length=True,
                                    )['input_ids'].to(device)


In [5]:
model.generate(input_image_tensor,outputs)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1024 but got size 32100 for tensor number 1 in the list.

In [7]:
outputs.shape

torch.Size([1, 1024])

In [7]:
from transformers import T5Tokenizer
tokenizer=T5Tokenizer.from_pretrained("google/flan-t5-base", bos_token = '<s>',add_bos_token = True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
tokenizer.eos_token_id

1

In [10]:
tokenizer.bos_token_id

32100

In [12]:
tokenizer.encode('<s> any thing </s>')

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:303: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


[32100, 136, 589, 1]